In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('titanic.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
temp = df.groupby('Pclass')['Age'].mean()
age_1 = temp[1]
age_2 = temp[2]
age_3 = temp[3]

In [5]:
def fill_age(row):
    if pd.isnull(row['Age']):
        if row['Pclass'] == 1:
            return age_1
        elif row['Pclass'] == 2:
            return age_2
        elif row['Pclass'] == 3:
            return age_3
    else:
        return row['Age']
df['Age'] = df.apply(fill_age, axis=1)

In [6]:
df['Embarked'] = df['Embarked'].fillna('S')

In [7]:
names = df['Name'].copy()

In [8]:
df.drop(['PassengerId', 'Cabin', 'Name', 'Ticket'], axis=1, inplace=True)

In [9]:
def convert(value):
    if value == 'male':
        return 1
    else:
        return 0
df['Sex'] = df['Sex'].apply(convert)

In [10]:
df[list(pd.get_dummies(df['Embarked']).columns)] = pd.get_dummies(df['Embarked'])

In [11]:
df.drop('Embarked', axis=1, inplace=True)

In [12]:
def convert(value):
    if value:
        return int(value)
df['Age'] = df['Age'].apply(convert)

In [13]:
df['Relative'] = df['SibSp'] + df['Parch']

In [14]:
def convert(value):
    if value > 0:
        return 1
    else:
        return 0
df['Relative'] = df['Relative'].apply(convert)

In [15]:
df.drop('Relative', axis=1, inplace=True)

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [27]:
x = df.drop('Survived', axis=1)
y = df['Survived']

In [43]:
x_train, x_test, y_train, y_test, names_train, names_test = train_test_split(x, y, names, test_size=0.25, random_state=42)

In [44]:
sc = StandardScaler()

In [45]:
x_train = sc.fit_transform(x_train)

In [46]:
x_test = sc.transform(x_test)

In [47]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x_train,y_train)

KNeighborsClassifier()

In [48]:
preds = classifier.predict(x_test)

In [49]:
print(f"Confusion matrix: \n{confusion_matrix(y_test,preds)}")

Confusion matrix: 
[[113  21]
 [ 21  68]]


In [50]:
print(f"Accuracy scorre: \n{accuracy_score(y_test,preds)}")

Accuracy scorre: 
0.8116591928251121


In [51]:
results = pd.DataFrame({
    "Name":names_test,
    "Actual":y_test.values,
    "Predicted":preds,
})

In [52]:
print("Survived and Predicted:")
survived_correct = results[(results['Actual'] == 1) & (results['Predicted'] == 1)]
print(f"Survived correct: {len(survived_correct)}")
for name in survived_correct['Name'].values:
    print(f"Name: {name}")

Survived and Predicted:
Survived correct: 68
Name: Harper, Miss. Annie Jessie "Nina"
Name: Barber, Miss. Ellen "Nellie"
Name: Kelly, Miss. Anna Katherine "Annie Kate"
Name: Carr, Miss. Helen "Ellen"
Name: Newsom, Miss. Helen Monypeny
Name: Lines, Miss. Mary Conover
Name: O'Leary, Miss. Hanora "Norah"
Name: Seward, Mr. Frederic Kimber
Name: Nysten, Miss. Anna Sofia
Name: Madigan, Miss. Margaret "Maggie"
Name: Serepeca, Miss. Augusta
Name: Baclini, Miss. Marie Catherine
Name: Hays, Mrs. Charles Melville (Clara Jennings Gregg)
Name: Carter, Master. William Thornton II
Name: Taussig, Miss. Ruth
Name: Baxter, Mrs. James (Helene DeLaudeniere Chaput)
Name: Cameron, Miss. Clear Annie
Name: Devaney, Miss. Margaret Delia
Name: Mellinger, Miss. Madeleine Violet
Name: Clarke, Mrs. Charles V (Ada Maria Winfield)
Name: Stephenson, Mrs. Walter Bertram (Martha Eustis)
Name: Ryerson, Miss. Emily Borie
Name: Behr, Mr. Karl Howell
Name: Compton, Miss. Sara Rebecca
Name: Frolicher, Miss. Hedwig Margaritha

In [54]:
print("Not Survived and Predicted:")
survived_correct = results[(results['Actual'] == 0) & (results['Predicted'] == 0)]
print(f"Survived correct: {len(survived_correct)}")
for name in survived_correct['Name'].values:
    print(f"Name: {name}")

Not Survived and Predicted:
Survived correct: 113
Name: Kvillner, Mr. Johan Henrik Johannesson
Name: Alhomaki, Mr. Ilmari Rudolf
Name: Vander Planke, Mr. Leo Edmondus
Name: Futrelle, Mr. Jacques Heath
Name: Kelly, Mr. James
Name: Lefebre, Miss. Jeannie
Name: Attalah, Mr. Sleiman
Name: Fox, Mr. Stanley Hubert
Name: Montvila, Rev. Juozas
Name: Porter, Mr. Walter Chamberlain
Name: Mineff, Mr. Ivan
Name: Olsvigen, Mr. Thor Anderson
Name: Risien, Mr. Samuel Beard
Name: Skoog, Master. Karl Thorsten
Name: Uruchurtu, Don. Manuel E
Name: Skoog, Master. Harald
Name: Petroff, Mr. Nedelio
Name: Hoyt, Mr. William Fisher
Name: Harknett, Miss. Alice Phoebe
Name: Yasbeck, Mr. Antoni
Name: Nankoff, Mr. Minko
Name: Hickman, Mr. Stanley George
Name: Braund, Mr. Lewis Richard
Name: Sirota, Mr. Maurice
Name: Keane, Mr. Andrew "Andy"
Name: Ford, Mr. William Neal
Name: Crease, Mr. Ernest James
Name: Odahl, Mr. Nils Martin
Name: Ali, Mr. Ahmed
Name: Newell, Mr. Arthur Webster
Name: Skoog, Mr. Wilhelm
Name: Du

In [55]:
print("Survived and Predicted incorrectly")
survived_correct = results[(results['Actual'] == 1) & (results['Predicted'] == 0)]
print(f"Survived correct: {len(survived_correct)}")
for name in survived_correct['Name'].values:
    print(f"Name: {name}")

Survived and Predicted incorrectly
Survived correct: 21
Name: Moubarek, Master. Halim Gonios ("William George")
Name: Nicola-Yarred, Miss. Jamila
Name: Kimball, Mr. Edwin Nelson Jr
Name: Andersen-Jensen, Miss. Carla Christine Nielsine
Name: Wilhelms, Mr. Charles
Name: Cohen, Mr. Gurshon "Gus"
Name: Sloper, Mr. William Thompson
Name: Taylor, Mr. Elmer Zebley
Name: Dahl, Mr. Karl Edwart
Name: de Mulder, Mr. Theodore
Name: Blank, Mr. Henry
Name: Homer, Mr. Harry ("Mr E Haven")
Name: Moubarek, Master. Gerios
Name: Emanuel, Miss. Virginia Ethel
Name: Yasbeck, Mrs. Antoni (Selini Alexander)
Name: Bradley, Mr. George ("George Arthur Brayton")
Name: Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)
Name: Woolner, Mr. Hugh
Name: Sheerlinck, Mr. Jan Baptist
Name: Barah, Mr. Hanna Assi
Name: Carter, Mr. William Ernest


In [56]:
print("Survived and Predicted incorrectly")
survived_correct = results[(results['Actual'] == 0) & (results['Predicted'] == 1)]
print(f"Survived correct: {len(survived_correct)}")
for name in survived_correct['Name'].values:
    print(f"Name: {name}")

Survived and Predicted incorrectly
Survived correct: 21
Name: Olsson, Miss. Elina
Name: Barbara, Mrs. (Catherine David)
Name: Mack, Mrs. (Mary)
Name: Lahtinen, Mrs. William (Anna Sylfven)
Name: Mangan, Miss. Mary
Name: Funk, Miss. Annie Clemmer
Name: Rosblom, Mrs. Viktor (Helena Wilhelmina)
Name: Artagaveytia, Mr. Ramon
Name: Danoff, Mr. Yoto
Name: Farthing, Mr. John
Name: Goldschmidt, Mr. George B
Name: Ford, Mrs. Edward (Margaret Ann Watson)
Name: Giglio, Mr. Victor
Name: Arnold-Franchi, Mrs. Josef (Josefine Franchi)
Name: Carter, Mrs. Ernest Courtenay (Lilian Hughes)
Name: Bourke, Mrs. John (Catherine)
Name: Boulos, Miss. Nourelain
Name: Tikkanen, Mr. Juho
Name: Baxter, Mr. Quigg Edmond
Name: Cacic, Miss. Marija
Name: Yrois, Miss. Henriette ("Mrs Harbeck")
